In [2]:
import pandas as pd
import spacy 
nlp = spacy.load("de_core_news_sm")
import matplotlib.pyplot as plt
import gzip,os,glob
import numpy as np
import pickle
#import librosa
from sklearn import *
from math import floor

In [3]:
def generate_random_vector(dimensions,seed):
    arr = np.array([0] * (dimensions-seed) + [1] * int(seed/2) + [-1] * int(seed/2) )
    np.random.shuffle(arr)
    return(arr)

In [4]:
import csv
import re
textz=[]
word_vec=dict()
word_docz=dict()
dimensions=500
seed=8

with open(r'C:\Users\Johannes\Documents\Projekte\Code\udk_library_local/data_movies_content_after_nltk_rri.csv', newline='', encoding="utf8") as csvfile:
    reader = csv.DictReader(csvfile) 
    for row in reader:
        textz.append(row['content'].lower())
        #if int(row['id'])>5000:
        #    break
        if int(row['id'])%1000==0:
            print(row['id'])
        current_doc_wordz=dict()
        content=row['content'].lower()
        for word in re.split('[\W]', content):
            if word not in current_doc_wordz:
                current_doc_wordz[word]=1
                try:
                    word_docz[word]=word_docz[word]+1
                except:
                    word_docz[word]=1
                    word_vec[word]=generate_random_vector(dimensions,seed)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000


In [6]:
#we focus on words which occur in more than HAPAX_THRESHOLD and less than STOPWORD_THRESHOLD documents because they just bring noise in our vector space

HAPAX_THRESHOLD=4
STOPWORD_THRESHOLD=1000
nonsingular_wordz = dict((k, v) for k, v in word_docz.items() if v > HAPAX_THRESHOLD and v<STOPWORD_THRESHOLD)


#we create IDF (inverse-document frequency) weights
normalize_factor=1 / sum(nonsingular_wordz.values())
for k in nonsingular_wordz:
    nonsingular_wordz[k]*=normalize_factor
    #print(nonsingular_wordz[k],k)
    
len(nonsingular_wordz)

19327

In [7]:
#0th pass - from random word vectors to meaningful doc vectors
print("rand word 2 doc\n")
doc_vec=np.zeros((len(textz),dimensions))
line_id=0
for text in textz:
    for word in re.split(' ',text):
        if word in nonsingular_wordz:
            #document vector is a linear combination of IDF-weighted word vectors which document contains
            doc_vec[line_id] = ( word_vec[word] / (nonsingular_wordz[word]) ) + doc_vec[line_id]
            #simplified version (withoug IDF-weighting)
            #doc_vec[line_id] = word_vec[word]   + doc_vec[line_id]
    line_id=line_id+1
 

for i in range(len(doc_vec)):
    doc_vec[i]=preprocessing.normalize(doc_vec[i].reshape(1,-1))

    
#from document vectors to word vectors
print("doc 2 word")
word_vec2=dict()
line_id=0
for text in textz:
    for word in re.split(' ',text):
        if word in nonsingular_wordz:
            #print(word)
            if word in word_vec2:
                word_vec2[word] +=  doc_vec[line_id] 
            else:
                word_vec2[word]=np.array([0.0]*dimensions)
                #print(str(word_vec2[word]))

                #doc_vec[line_id][0:18149]
    line_id=line_id+1

    
#normalize all word vectors
for k in word_vec2.keys():
    word_vec2[k]=preprocessing.normalize(word_vec2[k].reshape(1,-1))

print("word 2 doc")  
#and to doc vectors again
doc_vec2=np.zeros((len(textz),dimensions))
line_id=0
for text in textz:
    for word in re.split('[\W]',text):
        if word in nonsingular_wordz:
            #document vector is a linear combination of IDF-weighted word vectors which document contains
            try:
                doc_vec2[line_id] = ( word_vec2[word] / (nonsingular_wordz[word]) ) + doc_vec2[line_id]
            except:
                1
    line_id=line_id+1
    

rand word 2 doc

doc 2 word
word 2 doc


In [8]:
for i in range(len(doc_vec)):
    doc_vec2[i]=preprocessing.normalize(doc_vec2[i].reshape(1,-1))

    
#from document vectors to word vectors
print("doc2word 2")
word_vec2=dict()
line_id=0
for text in textz:
    for word in re.split(' ',text):
        if word in nonsingular_wordz:
            #print(word)
            if word in word_vec2:
                word_vec2[word] +=  doc_vec2[line_id] 
            else:
                word_vec2[word]=np.array([0.0]*dimensions)
                #print(str(word_vec2[word]))

                #doc_vec[line_id][0:18149]
    line_id=line_id+1

    
#normalize all word vectors
for k in word_vec2.keys():
    word_vec2[k]=preprocessing.normalize(word_vec2[k].reshape(1,-1))

print("word 2 doc 2")  
#and to doc vectors again
doc_vec2=np.zeros((len(textz),dimensions))
line_id=0
for text in textz:
    for word in re.split('[\W]',text):
        if word in nonsingular_wordz:
            #document vector is a linear combination of IDF-weighted word vectors which document contains
            try:
                doc_vec2[line_id] = ( word_vec2[word] / (nonsingular_wordz[word]) ) + doc_vec2[line_id]
            except:
                1
    line_id=line_id+1
    

doc2word 2
word 2 doc 2


In [9]:
from scipy.spatial import distance

def nearest_neighbor(query_id,database):
    query=database[query_id]
    closest = distance.cdist([query], database, 'cosine')
    return closest

In [10]:
import numpy.ma as ma

query_id=420
nns=nearest_neighbor(query_id,doc_vec)
nn_id=np.argmin(ma.masked_where(nns<0.0000000001, nns))
print(query_id,textz[query_id])
print(nn_id,nns[0][nn_id],textz[nn_id])

420 poetologie erinnern ernst meister spätwerk büchnerpreisträger ernst meister gelten lyriker jahrhundert gedicht ußerstes vervollständigung sicht zug infragestellung möglichkeitsbedingung sprechen grenze zusammenhang sprachverlust vergessen hintergrund kommen akt erinnern vielzahl bezugnahmen tradition vollziehen bedeutung aneignung hölderlin dienen pflege gedächtnis selbstvergewisserung mensch tod gott nietzsche conditio conditio poetica basis einzelanalysen gedicht machen karin herrmann entwicklungslogik werk perspektive ernst meister
12167 0.639871060223846 kreativität rückrufaktion kreativität lehren kulturwissenschaft alltag expertengremium möglichkeitsbedingung symbolkompetenter kreativindustrien kritikerinnen forderung kreativität gouvernementalen imperativ werk themaenheft fallstudium stadt kreativität kulturwissenschaft diskussionsschwerpunkt geniediskurs kreativitätsbegriff entstehungskontexte cultural studie identitätsrepräsentation internet kreativität fetischismus kultur

<h1>Compute Cosine Similarity through dot product</h1>

In [11]:
from sklearn.preprocessing import Normalizer
nrm = Normalizer('l2')

doc_vec_normed = nrm.fit_transform(doc_vec)

In [12]:
df = pd.read_pickle('../data/after_nltk/data_movies_after_nltk.pkl')
df

,title,author,content_nltk,medium
0,aernout mik communitas,NaN,aernout mik communitas aernout mik zählt zu de...,print
1,anton henning dt engl,NaN,anton henning dt engl die vorliegende publikat...,print
2,die feuchtwangers familie tradition und jüdis...,specht heike,die feuchtwangers familie tradition und jüdisc...,print
3,grüne philosophie ein konservativer denkansatz,scruton roger,grüne philosophie ein konservativer denkansatz...,print
4,technische dokumentation im maschinen und anl...,schlagowski heinz,technische dokumentation im maschinen und anla...,print
...,...,...,...,...
25587,jochen roller let's dance! lecture performance,Konzept und Leitung: Gabriele Brandstetter.,jochen roller let's dance! lecture performan...,movie
25588,ai weiwei - stimmen gegen das verstummen,ein Film von Birgit Adler-Conrad.,ai weiwei - stimmen gegen das verstummen der c...,movie
25589,new york tomorrow,réalisation: Yoann Le Gruiec. Scénario: Jean-M...,new york tomorrow die webdokumentation new yor...,movie
25590,hostess,ein Film von Rolf Römer. Mit: Annekathrin Bürg...,hostess eine hostess die seit zwei jahren mit ...,movie


In [13]:
# choose example 
example = query_id
title_1 = df['title'].iloc[example]
abstract_1 = textz[example]
# get example vector
vec_1 = doc_vec_normed[example,:]

# compute similarity
sim = np.dot(doc_vec_normed[example+1:,:],vec_1)

# find most similar document, see paper
title_2 = df['title'].iloc[np.argmax(sim)+example+1]
abstract_2 = textz[np.argmax(sim)+example+1]

print('\nSimilarity:')
print(np.max(sim))
print('\nTitle 1:')
print(title_1)
print('\nAbstract 1:')
print(abstract_1)
print('\nTitle 2:')
print(title_2)
print('\nAbstract 2:')
print(abstract_2)


Similarity:
0.3601289397761542

Title 1:
 kleines schopenhauerlexikon

Abstract 1:
poetologie erinnern ernst meister spätwerk büchnerpreisträger ernst meister gelten lyriker jahrhundert gedicht ußerstes vervollständigung sicht zug infragestellung möglichkeitsbedingung sprechen grenze zusammenhang sprachverlust vergessen hintergrund kommen akt erinnern vielzahl bezugnahmen tradition vollziehen bedeutung aneignung hölderlin dienen pflege gedächtnis selbstvergewisserung mensch tod gott nietzsche conditio conditio poetica basis einzelanalysen gedicht machen karin herrmann entwicklungslogik werk perspektive ernst meister

Title 2:
 kreativität eine rückrufaktion 

Abstract 2:
kreativität rückrufaktion kreativität lehren kulturwissenschaft alltag expertengremium möglichkeitsbedingung symbolkompetenter kreativindustrien kritikerinnen forderung kreativität gouvernementalen imperativ werk themaenheft fallstudium stadt kreativität kulturwissenschaft diskussionsschwerpunkt geniediskurs kreativit

<h1>K nearest Neighbors</h1>

In [14]:
# compares similarity of two abstracts through dot product

# validation_vector1 
validation_vector1 = query_id
val_abstract_1 = textz[validation_vector1]
# get validation_vector1
val_vec_1 = doc_vec_normed[validation_vector1,:]

# compute similarity of all entities
vec_temp = []
for vec in doc_vec_normed:
    i = np.dot(val_vec_1,vec)
    vec_temp.append(i)

# turn list into DataFrame
df_vec_temp = pd.DataFrame(vec_temp)

# Create new DataFrame for sorted sim-values
df_vec_temp_sorted = df_vec_temp.copy()
df_vec_temp_sorted.sort_values(by=[0],inplace = True, ascending=False)

# Get list of indices
lst_vec_temp_sorted = df_vec_temp_sorted.iloc[1:11,:].index.values.tolist()
lst_vec_temp_sorted

print(df['title'].iloc[validation_vector1])
print('\n')

for i in lst_vec_temp_sorted:
#    print('\n')
    print(df_vec_temp.iloc[i].values)
    print(df['title'].iloc[i])
#     print(textz[i])
#     print('\n')

 kleines schopenhauerlexikon


[0.36012894]
 kreativität eine rückrufaktion 
[0.3242632]
 ganz ohr neue musik in der gehörbildung 
[0.29305201]
 besucherbindung im kulturbetrieb ein handbuch
[0.28472913]
 seine welt wissen enzyklopädien in der frühen neuzeit 
[0.28127997]
 fahrleitungen elektrischer bahnen planung berechnung ausführung betrieb
[0.27391364]
 der himmel ist kein ort roman
[0.27080921]
 lucien clergue brasília clergue lucien brasilia eclergue lucien brasilia 
[0.24808826]
 transzendenz und negativität religionsphilosophische und ästhetische studien
[0.2075674]
cast.
[0.19010765]
 das jüdische in den wochenzeitungen zeiz spiegell und stern 19461989 berichterstattung zwischen popularisierungsbemühung vereinnahmung und abwehr


In [15]:
np.save('../data/after_nltk/rri_docvecs_for_tsne', doc_vec)